In [205]:
from scipy.stats.qmc import LatinHypercube as LHSampler
from matplotlib import pyplot as plt
from typing import List, Tuple
import numpy as np
import copy

In [285]:
LHS = LHSampler(d=3)

In [324]:
def lhs_grader(lhs: np.ndarray):
    y = sudoku_identity(lhs)
    # print([np.min(y[s]) for s in range(len(y))])
    # reducing y
    grade = np.min([np.sum(y[:, s]) for s in range(y.shape[1])])
    return grade

def sudoku_identity(lhs: np.array):
    N, P = lhs.shape
    y = np.zeros(lhs.shape, dtype=int)
    # y = [[0 for _ in range(len(lhs))] for i in range(len(lhs))]
    for q in range(N):  # find another way out pls
        interval = [q/N, (q+1)/N]
        for j in range(P):
            for i in range(N):
                if(lhs[i, j] is not None and interval[0] <= lhs[i, j] <= interval[1]):
                    y[q, j] = int(1)
                    break
    return y


# auxiliary h-bar matrix 
def lhs_grader_h(lhs: np.ndarray):
    y = sudoku_identity(lhs)
    # print([np.min(y[s]) for s in range(len(y))])
    # reducing y
    grade = np.min([np.sum(y[:, s, 0]) for s in range(y.shape[1])])
    return grade


def sudoku_identity_h(lhs: np.array):
    N, P = lhs.shape
    y = np.zeros((*lhs.shape, 2), dtype=int)
    # y = [[0 for _ in range(len(lhs))] for i in range(len(lhs))]
    for q in range(N):  # find another way out pls
        interval = [q/N, (q+1)/N]
        for j in range(P):
            y[q, j, 1] = -1
            for i in range(N):
                if(lhs[i, j] is not None and interval[0] <= lhs[i, j] <= interval[1]):
                    y[q, j, 0], y[q, j, 1] = int(1), i
                    break
    return y

In [353]:
def anti_sudoku_identity(lhs: np.array):
    N, P = lhs.shape
    y = np.zeros(lhs.shape, dtype=int)
    for q in range(N):
        interval = [q/N, (q+1)/N]
        for j in range(P):
            for i in range(N):
                if(lhs[i, j] is not None and interval[0] <= lhs[i, j] <= interval[1]):
                    y[q, j] += int(1)
    return y

In [358]:
# PeLHS (Partial expanded LHS)

def eLHS(lhs: np.ndarray, M:int):
    N, P = lhs.shape

    if(lhs_grader(lhs) == N):
        raise ValueError("Parameter lhs must be a proper LHS sample")
    

    exp = {"N": N,
           "M": M,
           "F": None,
           "Q": None,
           "expansion": np.array([]), 
           "grade": None }

    

ss1 = LHS.random(10)
# ss1[2] = ss1[3] + ss1[1] # interesting test
ss1[2] = LHS.random(1)
# print(ss1[2])
# print(sudoku_identity(ss1))
print(anti_sudoku_identity(ss1))
lhs_grader(ss1)



[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 2 1]
 [1 1 1]
 [1 0 1]
 [0 1 1]
 [2 1 2]
 [1 1 0]]


9

In [224]:
# SANDBOX
# tick for PeLHS vacancies 
def PartialeLHS(lhs: np.ndarray, M:int):
    # vacancies = [[i for i in range(lhs.shape[0] + M)] for _ in range(lhs.shape[1])]
    N, P = lhs.shape
    PeLHS = np.concatenate((lhs, [[None for _ in range(P)] for _ in range(M)]))
    return sudoku_identity(PeLHS)
    
N, M = 123, 87
ss2 = LHS.random(N)
pelhs = PartialeLHS(ss2, M)
print("# of zeroes: ", N + M - np.sum(pelhs[:, 0]))
print("# of zeroes: ", N + M - np.sum(pelhs[:, 1]))
print("# of zeroes: ", N + M - np.sum(pelhs[:, 2]))

# pippo = np.array([*np.indices([3]), *np.indices([3])])


# of zeroes:  92
# of zeroes:  97
# of zeroes:  94
